<a href="https://colab.research.google.com/github/ricopgm/Projeto_Imersao_IA/blob/main/Proj_Imersao_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
# Instalando o sdk do google generative ai
!pip install -q -U google-generativeai

In [ ]:
# Instalando biblioteca para manipulação e análise de dados.
!pip install pandas

In [ ]:
# Intalando biblioteca para criação de gráficos estáticos
!pip install matplotlib

In [ ]:
# Instalando biblioteca para criação de elementos interativos (widgets) em notebooks Jupyter (como o Google Colab)
!pip install ipywidgets

In [ ]:
# Instalando biblioteca para criação de gráficos interativos.
!pip install plotly

In [ ]:
# Instalando biblioteca para extração de dados de arquivos PDF
!pip install tabula-py

In [ ]:
# Instalando biblioteca para leitura e manipulação de arquivos PDF
!pip install PyPDF2

In [37]:
# Fazendo o import das bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import plotly.express as px
from tabula import read_pdf
from PyPDF2 import PdfReader

In [38]:
# Configurando API KEY como secreta
import google.generativeai as genai
from google.colab import userdata
api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

In [ ]:
# Função para fazer upload de arquivo de dados gerados pelo Gemini
def upload_arquivo():
  uploaded = widgets.FileUpload(accept='.xlsx, .csv, .pdf', multiple=False)
  display(uploaded)
  return uploaded

# Função para input de dados manualmente
def input_manual():
  print("Insira os dados manualmente, separados por vírgula, seguindo o formato:")
  print("Data Pedido,Previsão Entrega,Data Entrega,Cliente,idVeiculo,Nota Fiscal,Peso Líquido,Peso Cubado,Valor Mercadoria,Valor Frete Líquido,Devolução,Status")
  data = input()
  return data

# Função para carregar arquivo
def carregar_arquivo(uploaded):
  if uploaded.value:
    tipo_arquivo = uploaded.metadata[0]['type']
    if tipo_arquivo == 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet':
      df = pd.read_excel(uploaded.value[0]['content'])
    elif tipo_arquivo == 'text/csv':
      df = pd.read_csv(uploaded.value[0]['content'])
    elif tipo_arquivo == 'application/pdf':
      df = read_pdf(uploaded.value[0]['content'], pages='all')[0]
    else:
      print("Tipo de arquivo inválido. Por favor, faça upload de um arquivo .xlsx, .csv ou .pdf.")
      return None
    return df
  else:
    print("Nenhum arquivo selecionado. Iniciando input manual.")
    data = input_manual()
    df = pd.DataFrame([x.split(',') for x in data.split('\n')])
    df.columns = df.iloc[0]
    df = df[1:]
    return df

# Função para processar os dados do DataFrame
def processar_dados(df):
  # Renomear colunas se necessário
  df.columns = ['Data Pedido', 'Previsão Entrega', 'Data Entrega', 'Cliente', 'Código de Veículo',
                'Nota Fiscal', 'Peso Líquido', 'Peso Cubado', 'Valor Mercadoria', 'Valor Frete Líquido',
                'Devolução', 'Status']

  # Converter colunas de datas para o tipo datetime
  df["Data Pedido"] = pd.to_datetime(df["Data Pedido"], format="%d/%m/%Y")
  df["Previsão Entrega"] = pd.to_datetime(df["Previsão Entrega"], format="%d/%m/%Y")
  df["Data Entrega"] = pd.to_datetime(df["Data Entrega"], format="%d/%m/%Y")

  # Converter a coluna "Valor Mercadoria" para float
  df["Valor Mercadoria"] = df["Valor Mercadoria"].str.replace("R\$ ", "", regex=True).str.replace(".", "", regex=True).str.replace(",", ".", regex=True).astype(float)

  # Converter a coluna "Valor Frete Líquido" para float
  df["Valor Frete Líquido"] = df["Valor Frete Líquido"].str.replace("R\$ ", "", regex=True).str.replace(".", "", regex=True).str.replace(",", ".", regex=True).astype(float)

  # Criar coluna "Ano"
  df["Ano"] = df["Data Pedido"].dt.year

  # Criar coluna "Trimestre"
  df["Trimestre"] = df["Data Pedido"].dt.quarter

  # Criar coluna "Entregas Atrasadas"
  df["Entregas Atrasadas"] = df["Data Entrega"] > df["Previsão Entrega"]
  return df

# Funções para gerar os gráficos
def grafico_barras_verticais(df, coluna_x, coluna_y, titulo):
  fig = px.bar(df, x=coluna_x, y=coluna_y, title=titulo)
  fig.show()

def grafico_barras_horizontais(df, coluna_x, coluna_y, titulo):
  fig = px.bar(df, x=coluna_x, y=coluna_y, title=titulo, orientation='h')
  fig.show()

def grafico_rosca(df, coluna, titulo):
  fig = px.pie(df, values=coluna, names=coluna, title=titulo)
  fig.update_traces(textposition='inside', textinfo='percent+label')
  fig.show()

# Função para criar o dashboard
def criar_dashboard(df):
  # Widgets para seleção de período
  inicio_periodo = widgets.DatePicker(description='Data Inicial:')
  fim_periodo = widgets.DatePicker(description='Data Final:')

  # Função para atualizar os gráficos com base no período selecionado
  def atualizar_graficos(inicio, fim):
    # Filtrar dados pelo período
    fretes_filtrado = df[(df["Data Pedido"] >= inicio) & (df["Data Pedido"] <= fim)]

    # Gráfico de barras verticais - Receita Bruta por Ano
    grafico_barras_verticais(fretes_filtrado.groupby("Ano")["Valor Mercadoria"].sum().reset_index(), "Ano", "Valor Mercadoria", "Receita Bruta por Ano")

    # Gráfico de barras horizontais - Receita Bruta por Trimestre
    grafico_barras_horizontais(fretes_filtrado.groupby("Trimestre")["Valor Mercadoria"].sum().reset_index(), "Trimestre", "Valor Mercadoria", "Receita Bruta por Trimestre")

    # Gráfico de rosca - Entregas Atrasadas
    grafico_rosca(fretes_filtrado["Entregas Atrasadas"].value_counts().reset_index(), "Entregas Atrasadas", "Entregas Atrasadas")

    # Gráfico de barras verticais - Quantidade de Entregas por Status
    grafico_barras_verticais(fretes_filtrado["Status"].value_counts().reset_index(), "Status", "count", "Quantidade de Entregas por Status")

    # Gráfico de barras horizontais - Quantidade de Entregas por Baú do Veículo
    grafico_barras_horizontais(fretes_filtrado["Baú"].value_counts().reset_index(), "Baú", "count", "Quantidade de Entregas por Baú do Veículo")

    # Gráfico de rosca - Quantidade de Entregas por Tipo de Veículo
    grafico_rosca(fretes_filtrado["Tipo de Veículo"].value_counts().reset_index(), "Tipo de Veículo", "Quantidade de Entregas por Tipo de Veículo")

    # Gráfico de barras verticais - Quantidade de Entregas por Placa do Veículo
    grafico_barras_verticais(fretes_filtrado["Placa"].value_counts().reset_index(), "Placa", "count", "Quantidade de Entregas por Placa do Veículo")

    # Gráfico de barras horizontais - Devoluções por Baú do Veículo
    grafico_barras_horizontais(fretes_filtrado[fretes_filtrado["Devolução"] != "Sem Devolução"]["Baú"].value_counts().reset_index(), "Baú", "count", "Devoluções por Baú do Veículo")

    # Gráfico de rosca - Devoluções por Motivo
    grafico_rosca(fretes_filtrado[fretes_filtrado["Devolução"] != "Sem Devolução"]["Devolução"].value_counts().reset_index(), "Devolução", "Devoluções por Motivo")

  # Interação com os widgets
  widgets.interactive(atualizar_graficos, inicio=inicio_periodo, fim=fim_periodo)

  # Exibir os widgets
  display(inicio_periodo, fim_periodo)

# Solicitar o upload do arquivo
uploaded = upload_arquivo()

# Carregar o arquivo
df = carregar_arquivo(uploaded)

# Processar os dados
df = processar_dados(df)

# Criar o dashboard
criar_dashboard(df)